In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from pymystem3 import Mystem
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import string
import razdel
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm_notebook as tqdm

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

Using TensorFlow backend.


1.12.0


In [2]:
data = pd.read_csv('train.csv')

In [24]:
test = pd.read_csv('test.csv')

In [26]:
test.shape

(58788, 2)

In [4]:
def tokkenize_razdel(text):
    return [token.text for token in razdel.tokenize(text)]
 
def my_tokenizer(text):
    result = []
    for doc in text:    
        doc = tokkenize_razdel(doc)
        result.append(doc)        
    return result

In [5]:
X = np.array(data['text'])
Y = np.array(data['labels'])

In [9]:
pad_id = 0
oov_id = 'UNK'
index_offset = 2
sentence_size = 60
vocab_size = 5000
embedding_size = 20
model_dir = tempfile.mkdtemp()

def preprocess_vectorize(data):
    data = my_tokenizer(data)
    
    for element in data:
        element = ' '.join(element)

    tokenizer = Tokenizer(num_words = vocab_size, 
                          filters ='!"$%&()*+,-./:;=?@[\\]^_`{|}~\t\n<>', 
                          lower=True, 
                          oov_token= oov_id)
    tokenizer.fit_on_texts(data)
    data = tokenizer.texts_to_sequences(data)
    
    return data

In [11]:
%%time
X_variable = preprocess_vectorize(X)

CPU times: user 12.2 s, sys: 90.9 ms, total: 12.3 s
Wall time: 12.3 s


In [12]:
X_variable

[[1937,
  91,
  8,
  102,
  7,
  1,
  677,
  4214,
  695,
  14,
  4214,
  1,
  4018,
  4,
  24,
  39,
  275,
  51,
  81,
  2],
 [8,
  20,
  135,
  7,
  2818,
  1,
  56,
  677,
  21,
  695,
  14,
  1,
  10,
  1,
  55,
  4,
  24,
  93,
  275,
  51,
  81,
  2],
 [6, 15, 8, 29, 7, 44, 1, 2],
 [4, 36, 8, 11, 9, 29, 7, 288, 1, 2],
 [6, 11, 8, 200, 14, 94, 23, 301, 1584, 2],
 [6, 15, 8, 223, 92, 23, 301, 1584, 2],
 [4, 15, 65, 498, 827, 9, 74, 2],
 [6, 15, 16, 65, 498, 827, 9, 74, 2],
 [4, 63, 38, 109, 1152, 292, 45, 2],
 [4, 63, 38, 109, 1152, 292, 2],
 [4,
  49,
  7,
  101,
  1304,
  112,
  17,
  408,
  493,
  777,
  234,
  28,
  305,
  25,
  34,
  1368,
  149,
  473,
  2],
 [6, 50, 7, 101, 1304, 404, 3, 305, 1, 10, 117, 33, 1449, 9, 190, 10, 117, 2],
 [4, 24, 1, 112, 2],
 [4, 24, 1, 1396, 2],
 [18, 12, 69, 87, 72, 900, 3922, 206, 39, 12, 1, 2654, 25, 154, 2],
 [18, 11, 46, 333, 72, 206, 27, 15, 29, 2654, 879, 73, 154, 2],
 [18, 11, 1, 271, 285, 2],
 [18, 12, 1, 14, 1, 1, 285, 2],
 [6, 36, 